<a href="https://colab.research.google.com/github/DarekGit/FACES_DNN/blob/master/notebooks/05_05_TRAINING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

[Spis treści](https://github.com/DarekGit/FACES_DNN/blob/master/notebooks/Praca_Dyplomowa.ipynb) | [1. Wstęp](01_00_Wstep.ipynb) | [2. Metryki oceny detekcji](02_00_Miary.ipynb) | [3. Bazy danych](03_00_Datasety.ipynb) | [4. Przegląd metod detekcji](04_00_Modele.ipynb) | [5. Detekcja twarzy z wykorzystaniem wybranych architektur GSN](05_00_Modyfikacje.ipynb) | [6. Porównanie modeli](06_00_Porownanie.ipynb) | [7. Eksport modelu](07_00_Eksport_modelu.ipynb) | [8. Podsumowanie i wnioski](08_00_Podsumowanie.ipynb) | [Bibliografia](Bibliografia.ipynb)


---


## 5.5 Procedura trenowania sieci Faster R-CNN z MobileNetV2

W celu przyspieszenia procesu uczenia i wyboru hiperparametrów do modyfikacji i testów, rozdzielono notebooki na cześć trenującą oraz analizy wyników cząstkowych.
<br><br>Notebook trenujący odkłada wyniki cząstkowe do katalogu dedykowanego dla wybranej konfiguracji. Ze względu na rozmiar zebranych wyników przekraczający 50GB wykorzystano google drive.
<br>Przykład notebooka trenującego końcowy etap modelu 'BN_Mish_V2_250+F_2_50k': <br>
[ mobilenet_v2_Mish_V22F.ipynb](https://github.com/DarekGit/FACES_DNN/blob/master/notebooks/WIDERFACE_Detectron2_from_scratch_DD_mobilenet_v2_Mish_V22F.ipynb)

<br>Notebook testowy posiada zesłownikowany dostęp do wyników cząstkowych oraz narządzenia do analizy i porównania wyników.<br>
Notebook testujący: [ mobilenet_v2_test.ipynb](https://github.com/DarekGit/FACES_DNN/blob/master/notebooks/WIDERFACE_Detectron2_DD_mobilenet_v2_test.ipynb)

<br>W trakcie trenowaniu modelu dla wybranych parametrów modelu - wykonywano testy wyników cząstkowych. W przypadku osiągnięcia najlepszego wyniku dla danej konfiguracji - tworzono podkatalog z nowym notebookiem ze zmodyfikowanym zestawem parametrów sieci i rozpoczynano doszkalanie sieci od  etapu najlepszego wyniku poprzedniego treningu. Wyboru parametrów dokonano pod kątem optymalizacji wyników mAP50.
<br>W ten sposób powstało drzewo katalogów dla testów z kolejnymi notebookami dla zmodyfikowanych zestawów hiperparametrów. Każdy katalog zawiera notebook ze specyficznymi parametrami, pliki konfugraycyjne, wyniki cząstkowe, wizualizacje wyników.
<br>Dla przyspieszenia procesu trenowano kilka modeli jednocześnie.

<br>Trening rozpoczęto od parametrów domyślnych w repozytorium  Detectron2 z backbone MobilenetV2, nstępnie sprawdzono trening dla BN w dwóch wersjach, kolejnym etapem był trening dla BN z Mish w 3 wersjach, z następującymi podkatologami:

```
- MN2
- MN2_BN 
- MN2_BN_V2
- MN2_Mish 
- MN2_Mish_V2 
           - V22  
                  - V222
           - V22F 
           - V22F2
                  - V22F22
           - V22F3 
           - V22F4 
- MN2_Mish_V3 
           - V32 
```

 z następującymi modyfikacjami parametrów trenujących:

```
- MN2        FrozenBN LR 1e-3 GAMMA 0.5 SCHEDULER 40k,100k,200k,400k do 800k iteracji

- MN2_BN     Frozen -> Batch Normalization

- MN2_BN_V2  poprawiono normalizacje dla WiderFace GAMMA 0.3 SCHEDULER 50k,150k,250k,400k do 800k iteracji

- MN2_Mish   wprowadzono funkcję aktywacji Mish, BN GAMMa 0.3  SCHEDULER 50k,150k,250k,400k do 800k iteracji

- MN2_Mish_V2 LR 2e-3 GAMMA 0.3 SCHEDULER 150k,210k,270k,330k do 400k iteracji

           - V22   douczanie od 250k, LR = 4e-6 GAMMA 0.2 SCHEDULER 30k,50k do 70k iteracji

                  - V222 do 120k iteracji

           - V22F douczanie od 250k z FrozenBN LR = 4e-6 GAMMA 0.2 SCHEDULER 30k,50k do 100k iteracji

           - V22F2 douczanie od 250k z FrozenBN LR = 1e-3 GAMMA 0.2 SCHEDULER 30k,50k do 70k iteracji

                  - V22F22 do 300k

           - V22F3 douczanie od 250k z FrozenBN LR = 2e-5 GAMMA 0.2 SCHEDULER 30k,50k do 70k iteracji

           - V22F4 douczanie od 250k z FrozenBN LR = 1e-4 GAMMA 0.2 SCHEDULER 30k,50k do 70k iteracji

- MN2_Mish_V3 LR 5e-3 GAMMA 0.3 SCHEDULER 150k,210k,270k,330k do 400k iteracji

           - V32 douczanie od 80k z FrozenBN LR = 5e-4 GAMMA 0.1 SCHEDULER 25k,50k do 70k iteracji

```
Domyślnie  zastosowano:
* Agumentację
  * ResizeShortestEdge(short_edge_length=(640, 672, 704, 736), max_size=1333, sample_style='choice'),
  * RandomFlip(),
* Batch size: 2,
* Scheduler: WarmupMultiStepLR do 1000 iteracji

<br><b>Łączenie skatalogowano pomiary dla 14 zestawów hiperparametrów i przeanalizowano ponad 100 wyników pośrednich mAP w tym ich wizualizacje. W analizie zmian parametrów kierowano się optymalizacją mAP50.</b>
<br><br> 


### Wyniki

Jako punkt odniesienia zastosowano wyniki dla MN2 z parametrami domyślnymi dla Detectron2 dla 800 tysięcy iteracji:<br>
AP50 WIDER FACE - 43%, FACES DD 73%

<br>Dla BN wyniki poprawiono odpowiednio na 50% i 90% przy dwa razy mniejszej liczbie iteracji

<br>BN_V2 nie wniosła dodatkowej poprawy

<br>Funkcja aktywacji Mish - zasadaniczo nie poprawiła wyników, ale znacząco przyspiszono trening, porównywalny wynik uzyskano dla 200k iteracji

<br>Dla konfiguracji z funkcją aktywacji Mish_V2 uzyskano wyniki mAP50 na poziomie- 54% dla 250k, 92% dla 250k iteracji

<br><b>Ostatecznie uzyskano poprawę wyników mAP50 o 12pp dla WIDER FACE oraz 20pp dla FACES DD, przy 8krotnym przyspieszeniu procesu trenowania.</b>
<br><br>
```
- MN2         maks. mAP50 43% WIDER FACE 73% FACES DD dla 800k iteracji
- MN2_BN      maks. mAP50 50% WIDER FACE 90% FACES DD dla 800k iteracji
- MN2_BN_V2   test przeprowadzono do 500k iteracji bez poprawy wyniku
- MN2_Mish    maks WIDER FACE 50%, 90% FACES DD dla 200k iteracji
- MN2_Mish_V2 maks WIDER FACE 54,73%, 92,39% FACES DD 250k iteracji
           - V22  maks 52,57% 92,05% dla 70k iteracji
                  - V222 maks 53,65% 92% dla 120k iteracji - przerwany
           - V22F  maks 55,24%  92,52% dla 100k iteracji
           - V22F2 maks 55,34%  92,96% dla 50k iteracji
                  - V22F22 bez poprawy
           - V22F3 maks 55,23%  92,74% dla 10k iteracji
           - V22F4 maks 54,94% dla 10k,  92,64% dla 20k iteracji
- MN2_Mish_V3 maks 51,06%  90,49% dla 80k iteracji
           - V32 maks 52,76%  dla 20k, 91,78% dla 30k iteracji
```
<br>





<div align="center">
<img src="https://github.com/DarekGit/FACES_DNN/blob/master/Figures/Detectron2_Wider_Face_mAP_MN2_BN_Mish_V2_250k.png?raw=1" alt="mAP MishV2 WIDER 250k" align="center" width="1000" >
<img src="https://github.com/DarekGit/FACES_DNN/blob/master/Figures/Detectron2_Faces_DD_mAP_MN2_BN_Mish_V2_250k.png?raw=2" alt="mAP MishV2 FACESDD 250k" align="center" width="1000" > 
<br> 

Rys. 1. Przykładowe wizualizacje dla MN2_Mish_V2 250k iteracji

</div>


<br>


<div align="center">
<img src="https://github.com/DarekGit/FACES_DNN/blob/master/Figures/Detectron2_Wider_Face_mAP_MN2_BN_Mish_V2_250%2BF_2_50k.png?raw=1" alt="mAP MishV2 FrozenBN WIDER +50k" align="center" width="1000" >
<img src="https://github.com/DarekGit/FACES_DNN/blob/master/Figures/Detectron2_Faces_DD_mAP_MN2_BN_Mish_V2_250%2BF_2_50k.png?raw=2" alt="mAP MishV2 FrozenBN FACESDD +50k" align="center" width="1000" >
<br> 

Rys. 2. Przykładowe wizualizacje po douczeniu z FrozenBN V2 dla 50k iteracji.
</div>



<br>

Do dalszej oceny i porównania wybrano modele:

### Model '800k' 

Zastosowano parametry domyślne Detectron2 - jako punkt odniesienia do dalszych analiz dla backbone MobileNetV2.
<br>
* Agumentacja
  * ResizeShortestEdge(short_edge_length=(640, 672, 704, 736), max_size=1333, sample_style='choice'),
  * RandomFlip(),
* FrozenBatchNorm2d, 
* Batch size: 2, 
* Scheduler: WarmupMultiStepLR
* Bazowy LR: 0.001 z liniowym rozgrzewaniem (linear warm-up) do 1000 iteracji, zmniejszany dwukrotnie przy 40k,10k,200k,400k do 800k iteracji 
<br><br>

Zapis trenowania modelu został dołączony do pracy jako załącznik w pliku: 
[ mobilenet_v2_800k.ipynb]( https://github.com/DarekGit/FACES_DNN/blob/master/notebooks/WIDERFACE_Detectron2_from_scratch_DD_mobilenet_v2_800k.ipynb)
<br>

### Model 'BN_800k' 


Do modelu wprowadzono Batch Normalization - pokazanie wpływu BN przy małym batchu=2<br>

* Agumentacja
  * ResizeShortestEdge(short_edge_length=(640, 672, 704, 736), max_size=1333, sample_style='choice'),
  * RandomFlip(),
* <b>BatchNorm2d,</b> 
* Batch size: 2, 
* Scheduler: WarmupMultiStepLR
* Bazowy LR: 0.001 z liniowym rozgrzewaniem (linear warm-up) do 1000 iteracji, zmniejszany dwukrotnie przy 40k,10k,200k,400k do 800k iteracji 
<br><br>

Zapis trenowania modelu został dołączony do pracy jako załącznik w pliku: 
[ mobilenet_v2_BN.ipynb](https://github.com/DarekGit/FACES_DNN/blob/master/notebooks/WIDERFACE_Detectron2_from_scratch_DD_mobilenet_v2_BN.ipynb)
<br>

### Model 'BN_Mish_V2_250+F_2_50k' 

Uzyskano najlepszy wynik uczenia dla backbone z MobileNetV2 wykorzystano funkcję Mish, Batch Normalization dla 250k iteracji + 50k z FrozenBN.<br>

* Agumentacja
  * ResizeShortestEdge(short_edge_length=(640, 672, 704, 736), max_size=1333, sample_style='choice'),
  * RandomFlip(),
* <b>funkcję aktywacji Mish,</b> 
* BatchNorm2d,
* Batch size: 2, 
* Scheduler: WarmupMultiStepLR
* Bazowy LR: 0.002 z liniowym rozgrzewaniem (linear warm-up) do 1000 iteracji, zmniejszany 0.3 po 150k,210k  do 250k iteracji

- <b>douczanie:</b>
* <b>FrozenBatchNorm2d,</b>
* LR: 4e-6, zmniejszony pięciokrotnie po 30k do 50k iteracji
<br><br>

Zapis trenowania modelu został dołączony do pracy jako załącznik w pliku: 
[ mobilenet_v2_Mish_V22F2.ipynb](https://github.com/DarekGit/FACES_DNN/blob/master/notebooks/WIDERFACE_Detectron2_from_scratch_DD_mobilenet_v2_Mish_V22F2.ipynb)
<br>

### Model 'BN_Mish_V3_80+30k'

Uzyskano najszybsze uczenie dla backbone z MobileNetV1 - 110k iteracji.<br>
 
Zastosowane rozwiązanie wykorzystuję architekturę MobileNet V2 w backbone 
* Agumentacja
  * ResizeShortestEdge(short_edge_length=(640, 672, 704, 736), max_size=1333, sample_style='choice'),
  * RandomFlip(),
* <b>funkcję aktywacji Mish,</b> 
* BatchNorm2d,
* Batch size: 2, 
* Scheduler: WarmupMultiStepLR
* Bazowy LR: 0.005 z liniowym rozgrzewaniem (linear warm-up),  do 80k iteracji

- <b>douczanie:</b>
* LR: 5e-4, zmniejszony dziesięciokrotnie po 25k do 30k iteracji
<br><br>

Zapis trenowania modelu został dołączony do pracy jako załącznik w pliku: 
[ mobilenet_v2_Mish_V32.ipynb](https://github.com/DarekGit/FACES_DNN/blob/master/notebooks/WIDERFACE_Detectron2_from_scratch_DD_mobilenet_v2_Mish_V32.ipynb)
<br>



---

[Spis treści](https://github.com/DarekGit/FACES_DNN/blob/master/notebooks/Praca_Dyplomowa.ipynb) | [1. Wstęp](01_00_Wstep.ipynb) | [2. Metryki oceny detekcji](02_00_Miary.ipynb) | [3. Bazy danych](03_00_Datasety.ipynb) | [4. Przegląd metod detekcji](04_00_Modele.ipynb) | [5. Detekcja twarzy z wykorzystaniem wybranych architektur GSN](05_00_Modyfikacje.ipynb) | [6. Porównanie modeli](06_00_Porownanie.ipynb) | [7. Eksport modelu](07_00_Eksport_modelu.ipynb) | [8. Podsumowanie i wnioski](08_00_Podsumowanie.ipynb) | [Bibliografia](Bibliografia.ipynb)


---